In [9]:
import json
from typing import Optional
import requests
from dataclasses import dataclass

In [10]:
latitude = 52.52
longitude = 13.41
url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
response = requests.get(url)

In [11]:
print(response.json())

{'latitude': 52.52, 'longitude': 13.419998, 'generationtime_ms': 0.06401538848876953, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 38.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2024-05-25T07:30', 'interval': 900, 'temperature': 18.2, 'windspeed': 9.7, 'winddirection': 184, 'is_day': 1, 'weathercode': 2}}


In [12]:
return {
    "messageVersion": "1.0",
    "response": {
        "actionGroup": event['actionGroup'],
        "apiPath": event['apiPath'],
        "httpMethod": event['httpMethod'],
        "httpStatusCode": 200,
        "responseBody": {
            "application/json": {
                "body": response.json()
            }
        }
    }
}

SyntaxError: 'return' outside function (335647751.py, line 1)

In [ ]:
import json
from typing import Optional
import requests
from dataclasses import dataclass


def format_params(parameters):
    result = {}
    for param in parameters:
        result[param['name']]=param['value']
    return result
    
    
def lambda_handler(event: dict, context: LambdaContext):
    #Event: {'agent': {'alias': 'TSTALIASID', 'name': 'GetWeather', 'version': 'DRAFT', 'id': '68X4FYOEUN'}, 
    #   'sessionId': '022346938362354', 'sessionAttributes': {}, 'promptSessionAttributes': {}, 'inputText': 'beijing', 
    #   'apiPath': '/get_lat_long', 'httpMethod': 'GET', 'messageVersion': '1.0', 'actionGroup': 'get_weather', 
    #   'parameters': [{'name': 'place', 'type': 'string', 'value': 'beijing'}]}
    params = format_params(event['parameters'])
    print(params)
    if event['apiPath'] == '/get_lat_long':
        url = "https://nominatim.openstreetmap.org/search"
        params = {'q': params.get('place'), 'format': 'json', 'limit': 1}
        # 403 error
        # response = requests.get(url, params=params)
        # print(response)
        # response = response.json()
        response = [{
            'lat': '39.9057136',
            'lon': '116.3912972'
        }]
        if response:
            lat = response[0]["lat"]
            lon = response[0]["lon"]
            result = {"latitude": lat, "longitude": lon}
        else:
            result = None
        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": event['actionGroup'],
                "apiPath": event['apiPath'],
                "httpMethod": event['httpMethod'],
                "httpStatusCode": 200,
                "responseBody": {
                    "application/json": {
                        "body": json.dumps(result)
                    }
                }
            }
        }
    elif event['apiPath'] == '/get_weather':
        latitude = params.get('latitude')
        longitude = params.get('longitude')
        url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
        response = requests.get(url)
        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": event['actionGroup'],
                "apiPath": event['apiPath'],
                "httpMethod": event['httpMethod'],
                "httpStatusCode": 200,
                "responseBody": {
                    "application/json": {
                        "body": response.json()
                    }
                }
            }
        }
    else:
        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": event['actionGroup'],
                "apiPath": event['apiPath'],
                "httpMethod": event['httpMethod'],
                "httpStatusCode": 404,
                "responseBody": {
                    "application/json": {
                        "body": '{"err":"no such function"}'
                    }
                }
            }
        }

In [20]:
url = "https://nominatim.openstreetmap.org/search"
params = {'q': 'beijing', 'format': 'json', 'limit': 1}
response = requests.get(url, params=params,headers={'referer': "indie-test"})
print(response.json())
response = response.json()

In [23]:
print(response.json())

[{'place_id': 196154718, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 25248662, 'lat': '39.9057136', 'lon': '116.3912972', 'class': 'place', 'type': 'city', 'place_rank': 15, 'importance': 0.7344014383682882, 'addresstype': 'city', 'name': '北京市', 'display_name': '北京市, 东城区, 北京市, 100010, 中国', 'boundingbox': ['39.7457136', '40.0657136', '116.2312972', '116.5512972']}]
